# 체인 설정 구성
- 런타임에 사용자나 개발자가 체인 구성을 변경할 수 있다
- 아래 2가지 방식으로 쉽게 구성할 수 있다
    1. `configurable_fields`
        - 시스템 설정 값을 정의하는 필드. 이를 통해 시스템 성능과 기능을 최적화할 수 있다.
        - 일반적으로 설정파일, 환경변수 등으로 설정
        - 데이터베이스 설정, 로길, 보안, 성능 튜닝 매개변수 등을 설정할 때 쓰인다.
    1. `configurable_alternatives`
        - 필드와 달리, 대체 가능한 runnable들을 나열한 후, 런타임에 선택해서 사용한다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.chat_models import ChatOllama

model = ChatOllama().configurable_fields(
    model=ConfigurableField(
        id="model_name",
        name="Model of ollama",
        description="Model supported by ollama",
    )
)

In [5]:
model.invoke(
    "대한민국 수도는 어디야?",
    config={"configurable": {"model_name": "gemma:7b"}}
)

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T14:42:03.050969166Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6246532723, 'load_duration': 6001474918, 'prompt_eval_count': 36, 'prompt_eval_duration': 106580000, 'eval_count': 10, 'eval_duration': 93731000}, id='run-679b4186-e242-4696-9ab7-67200bdbc1d7-0')

In [6]:
model.invoke(
    "대한민국 수도는 어디야?",
    config={"configurable": {"model_name": "mistral:7b"}}
)

AIMessage(content=' 대한민국 수도는 서울입니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2024-06-20T14:42:24.681347871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4310613259, 'load_duration': 3995696622, 'prompt_eval_count': 18, 'prompt_eval_duration': 102759000, 'eval_count': 17, 'eval_duration': 122731000}, id='run-ea3d8fd8-6583-46d9-89aa-2f2b9297340d-0')

`with_config`로도 설정할 수 있다

In [7]:
model.with_config(configurable={"model_name": "gemma:7b"}).invoke("미국의 수도는 어디야?")

AIMessage(content='미국의 수도는 **워싱턴 D.C.**입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T14:45:07.247141348Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7113998546, 'load_duration': 6813190647, 'prompt_eval_count': 35, 'prompt_eval_duration': 105816000, 'eval_count': 16, 'eval_duration': 150111000}, id='run-1b74b500-0674-4f87-b93d-0981ff6f9af2-0')

In [10]:
prompt = PromptTemplate.from_template("{x} 보다 큰 수 하나 알려줘")
chain = prompt | model
print(chain.with_config(configurable={"model_name": "mistral:7b"}).invoke({"x": 1}))
print(chain.invoke({"x": 10}, config={"configurable": {"model_name": "gemma:7b"}}))

content=' 이 질문에 답하려면 추가적인 정보가 필요합니다. 특정 범위, 집합, 함수 등을 기준으로 큰 수를 찾고 싶으신 것인지 확인해 주시면 도와드리겠습니다. 예를 들어 "2보다 큰 수"는 3이라고 할 수 있습니다. 더 자세한 설명을 부탁합니다!' response_metadata={'model': 'mistral:7b', 'created_at': '2024-06-20T14:47:33.767109725Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1305416717, 'load_duration': 2208626, 'prompt_eval_count': 12, 'prompt_eval_duration': 102105000, 'eval_count': 138, 'eval_duration': 1069044000} id='run-81904ebf-d587-4033-8d0a-ee6fb2602d0d-0'
content='13입니다' response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T14:47:40.981464726Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7205987567, 'load_duration': 6961293974, 'prompt_eval_count': 37, 'prompt_eval_duration': 108202000, 'eval_count': 10, 'eval_duration': 90542000} id='run-0bd8ecaf-59f8-4131-942c-218a1bfeae3c-0'


# HubRunnable
- Langchain hub에 있는 프롬프트에 config를 추가하여 전환을 용이하게 할 수 있다

In [11]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="The Hub commit to pull from",
    )
)

In [12]:
# 아무런 지정이 없으면 rlm/rag-pompt에서 프롬프트를 가져온다
prompt.invoke({"question": "Hello", "context": "World"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Hello \nContext: World \nAnswer:")])

In [13]:
prompt.with_config(
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "hello"})

ChatPromptValue(messages=[HumanMessage(content='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: hello\n\nSUMMARY:')])

# Configurable Alternative: LLM
- LLM Runnable 객체를 configurable하게 하여, 여러 객체를 나열한 후 필요에 따라 모델을 선택하도록 한다.


In [17]:
model = ChatOllama(model="gemma:7b").configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gemma",
    mistral=ChatOllama(model="mistral:7b"),
)
prompt = PromptTemplate.from_template("{topic}에 대해 간단히 설명해주세요")
chain = prompt | model

In [18]:
chain.invoke({"topic": "뉴진스"})

AIMessage(content='뉴진스는 미국 뉴욕주에 위치한 도시입니다. 뉴진스는 뉴욕주에서 가장 큰 도시이며, 미국에서도 10번째로 큰 도시입니다. 뉴진스는 문화, 경제, 교육의 중심지로, 세계적으로 유명합니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T15:01:49.904402222Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6920230769, 'load_duration': 6134147215, 'prompt_eval_count': 39, 'prompt_eval_duration': 106062000, 'eval_count': 63, 'eval_duration': 627360000}, id='run-47a78748-ee7b-4840-bdc0-3b727614487b-0')

In [19]:
chain.with_config(configurable={"model": "mistral"}).invoke("뉴진스")

AIMessage(content=' 뉴 저지 (New Jersey)는 미국 동부의 하롱스트 상태 중 하나입니다. 이 상태는 북-남으로 긋힌 자세로 옆에 덴버라이어(Delaware), 덜라와렌(Delaware), 파닛시(Pennsylvania) 상태, 뉴욕(New York)과 함께 애틱 몰트 지역(Atlantic coastal plain)에 위치하고 있습니다.\n\n이 상태는 북부와 남부로 나누어집니다. 뉴 저지의 북부에는 경계가 허점없이 그어져 있는 뉴욕과의 영역, 대학교들이 많고 산업 영역이 발달한 지구(Greater New York City Area)가 위치하며 이 지역에서는 주요 언어로 영어와 스페인어, 그리고 일본어, 중국어 등 다양한 언어가 사용됩니다.\n\n뉴저지의 남부 지역에는 대학교들이 없으며, 산업이 발달되어 있지 않지만 자연경관이 매우 아름답습니다. 여기에는 아틱 시티(Atlantic City), 캐피톨 섬(Cape May) 등 많은 휴양지와 대중 관광지가 있습니다.\n\n뉴저지는 미국의 47번째 크기로 약 8,721km²로 지배되고 있는 주민들이 2,938만명을 잇고 있습니다. 뉴저지의 주요 도시들은 뉴어클랜드(Newark), 티웜슈위(Trenton)과 켄사스시티(Camden)입니다.\n\n또한, 뉴저지는 미국의 주요 문화적인 중심지로 여러 흥행 음악 및 영화가 출장되었으며 많은 유명한 사람들이 이 상태에서 태어나거나 자라왔습니다. 예를 들어, 마틴 스카와르스(Martin Scorsese), 브루스 시플리스(Bruce Springsteen), 존 그런(Jon Bon Jovi) 등.', response_metadata={'model': 'mistral:7b', 'created_at': '2024-06-20T15:02:28.061133178Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 98537134

In [21]:
llm = ChatOllama(model="gemma:7b")

prompt = PromptTemplate.from_template(
    "{country} 수도는 어디야?"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="capital",
    # 대체 가능한 prompt 객체 리스트 나열
    area=PromptTemplate.from_template("{country}의 면적을 얼마야?"),
    population=PromptTemplate.from_template("{country}의 인구수는 얼마야?"),
    eng=PromptTemplate.from_template("{input}를 영어로 번역해주세요"),
)

chain = prompt | llm

In [23]:
chain.invoke({"country": "대한민국"})

AIMessage(content='수도는 서울입니다. 서울은 대한민국의 수도로, 국민의 행정 중심지입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T15:07:52.135609743Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6447482621, 'load_duration': 6032936115, 'prompt_eval_count': 36, 'prompt_eval_duration': 106429000, 'eval_count': 26, 'eval_duration': 262138000}, id='run-1dfdb220-9312-4f01-a602-1da47dae4d4d-0')

In [25]:
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "대한민국"})

AIMessage(content='대한민국의 면적은 **238,741km²**입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T15:08:17.257304804Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 446451284, 'load_duration': 2584585, 'prompt_eval_count': 25, 'prompt_eval_duration': 102599000, 'eval_count': 21, 'eval_duration': 209380000}, id='run-92e219a1-48a5-4f4d-aef8-39a30a682f68-0')

In [26]:
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "대한민국"})

AIMessage(content='2023년 기준,대한민국의 인구수는 **51,282,986명**입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T15:08:27.472919705Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 552792182, 'load_duration': 2214332, 'prompt_eval_count': 24, 'prompt_eval_duration': 102626000, 'eval_count': 31, 'eval_duration': 314299000}, id='run-7de33755-5136-46e2-8c44-b54c1efa0d30-0')

In [28]:
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "대한민국"})

AIMessage(content='Republic of Korea</start_of_turn>', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-20T15:08:43.691870777Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 341444002, 'load_duration': 2679635, 'prompt_eval_count': 27, 'prompt_eval_duration': 103172000, 'eval_count': 11, 'eval_duration': 104634000}, id='run-92fa6c1b-111e-4071-bffc-63888d895317-0')

In [29]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.chat_models import ChatOllama

llama3 = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)

llm = ChatOllama(model="gemma:7b").configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gemma",
    mistral=ChatOllama(model="mistral:7b"),
    llama3 = ChatHuggingFace(llm=llama3)
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘." 
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="description",
    # 대체 가능한 prompt 객체 리스트 나열
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
)

chain = prompt | llm

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
chain.with_config(configurable={"prompt": "founder", "model": "llama3"}).invoke({"company": "Apple"})

AIMessage(content='😊\n\nApple Inc. was founded on April 1, 1976, by three individuals:\n\n1. **Steve Jobs** (1955-2011): A visionary entrepreneur, inventor, and designer who is widely recognized as the co-founder of Apple. Jobs is credited with transforming Apple into one of the most successful and innovative companies in the world.\n2. **Steve Wozniak** (born 1950): A computer engineer and programmer who designed the Apple I and Apple II personal computers. Wozniak is often referred to as "Woz" and is known for his humility and generosity.\n3. **Ronald Wayne** (born 1934): A friend of Jobs and Wozniak who helped them establish Apple Computer. Wayne sold his 10% stake in the company back to Jobs and Wozniak in 1977, and is no longer involved with the company.\n\nThese three individuals came together to create Apple Computer, which was initially headquartered in Jobs\' parents\' garage in Los Altos, California. The rest, as they say, is history! 📊', id='run-84411296-a831-4941-a35b-a20ea

In [31]:
chain.with_config(configurable={"prompt": "competitor", "model": "mistral"}).invoke({"company": "Apple"})

AIMessage(content=' Apple의 주요 경쟁사로는 Google, Samsung Electronics, Huawei, Xiaomi, LG Electronics, Microsoft (Windows Phone), Amazon (Kindle/Echo devices) 등이 있습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2024-06-20T15:12:52.87811417Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4628882075, 'load_duration': 3996747995, 'prompt_eval_count': 21, 'prompt_eval_duration': 102855000, 'eval_count': 58, 'eval_duration': 439232000}, id='run-1af52987-53e8-4a95-8d22-cbbf40ca17d6-0')

- 설정을 저장하여 재사용할 수 있다

In [32]:
llama3_competiter_chain = chain.with_config(configurable={
    "prompt": "competitor",
    "model": "llama3",
})

In [33]:
llama3_competiter_chain.invoke({"company": "Samsung"})

AIMessage(content="😊\n\nSamsung Electronics is a leading technology company, and it has several competitors in various markets. Here are some of the main competitors in different categories:\n\n**Smartphones:**\n\n1. Apple (iPhone)\n2. Huawei (P30, Mate, etc.)\n3. Xiaomi (Redmi, Mi, etc.)\n4. Oppo (Find, Reno, etc.)\n5. Vivo (X, V, etc.)\n\n**Semiconductors:**\n\n1. Intel (processors, memory chips)\n2. Taiwan Semiconductor Manufacturing Company (TSMC) (processors, memory chips)\n3. Micron Technology (memory chips)\n4. SK Hynix (memory chips)\n5. Qualcomm (processors, modems)\n\n**TVs and Home Appliances:**\n\n1. LG Electronics (OLED TVs, washing machines, etc.)\n2. Sony Corporation (Bravia TVs, audio products, etc.)\n3. Vizio (smart TVs, soundbars, etc.)\n4. Hisense (smart TVs, refrigerators, etc.)\n5. TCL (smart TVs, air conditioners, etc.)\n\n**Wearables and IoT:**\n\n1. Fitbit (smartwatches, fitness trackers)\n2. Apple Watch (smartwatches)\n3. Huawei (smartwatches, fitness trackers)